In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms
import random
import time

VEHICLES_COST = 1000
UNATTENDED_CUSTOMER_COST = 10000
WAIT_DELAY_COST = 10

N_CUSTOMERS = 20
N_VEHICLES = 10

# Distance calculation between two customers
def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Fitness function to evaluate routes
def evaluate(individual):
    total_cost = 0
    wait_delay = 0
    all_customers = set(range(1, len(data)))  # All customers excluding depot
    assigned_customers = set()
    penalty_unattended = UNATTENDED_CUSTOMER_COST  # Penalización por cliente no atendido
    penalty_vehicle_usage = VEHICLES_COST  # Penalización por vehículo utilizado
    penalty_time_window = WAIT_DELAY_COST  # Penalización por minuto fuera de la ventana de tiempo

    vehicles_used = 0

    for vehicle_route in individual:
        if vehicle_route:  # Si el vehículo tiene una ruta asignada
            vehicles_used += 1

        current_time = 0
        last_customer = 0  # start at depot

        for customer in vehicle_route:
            if customer in assigned_customers:
                return float('inf'),  # Customer assigned multiple times

            distance = calculate_distance(data.loc[last_customer, 'X'], data.loc[last_customer, 'Y'], data.loc[customer, 'X'], data.loc[customer, 'Y'])
            total_cost += distance
            arrival_time = current_time + distance
            ready_time = data.loc[customer, 'READY_TIME']
            due_time = data.loc[customer, 'DUE_TIME']
            service_time = data.loc[customer, 'SERVICE_TIME']

            # Penalización por llegar antes o después de la ventana de tiempo
            if arrival_time < ready_time:
                wait_delay += max(0, ready_time - arrival_time) * penalty_time_window
            if arrival_time > due_time:
                wait_delay += max(0, arrival_time - due_time) * penalty_time_window

            current_time = max(arrival_time, ready_time) + service_time
            last_customer = customer
            assigned_customers.add(customer)

        # Return to depot
        total_cost += calculate_distance(data.loc[last_customer, 'X'], data.loc[last_customer, 'Y'], data.loc[0, 'X'], data.loc[0, 'Y'])

    # Penalización por clientes no atendidos
    if assigned_customers != all_customers:
        total_cost += penalty_unattended * (len(all_customers - assigned_customers))

    # Penalización por vehículos utilizados y tiempos de espera y delay
    total_cost += vehicles_used * penalty_vehicle_usage
    total_cost += wait_delay

    return total_cost,




# Generate a valid individual
def generate_individual():
    customers = list(range(1, len(data)))
    random.shuffle(customers)
    routes = [[] for _ in range(N_VEHICLES)]

    for i, customer in enumerate(customers):
        routes[i % 5].append(customer)

    return creator.Individual(routes)  # Ensure individual is a DEAP Individual

# Crossover function
def crossover(ind1, ind2):
    split1 = random.randint(1, len(ind1) - 1)
    split2 = random.randint(1, len(ind2) - 1)

    child1 = ind1[:split1] + ind2[split2:]
    child2 = ind2[:split2] + ind1[split1:]

    child1 = correct_duplicates(child1)
    child2 = correct_duplicates(child2)

    return creator.Individual(child1), creator.Individual(child2)

# Mutation function
def mutate(individual):
    if len(individual) > 1:
        vehicle1, vehicle2 = random.sample(range(len(individual)), 2)
        if len(individual[vehicle1]) > 0 and len(individual[vehicle2]) > 0:
            cust1 = random.choice(individual[vehicle1])
            cust2 = random.choice(individual[vehicle2])
            individual[vehicle1].remove(cust1)
            individual[vehicle2].remove(cust2)
            individual[vehicle1].append(cust2)
            individual[vehicle2].append(cust1)
    return individual,

# Fix duplicate customer assignments
def correct_duplicates(individual):
    all_customers = set(range(1, len(data)))
    flat_individual = [cust for route in individual for cust in route]

    if len(set(flat_individual)) < len(all_customers):
        missing_customers = list(all_customers - set(flat_individual))
        for route in individual:
            if len(route) > 0 and missing_customers:
                customer_to_replace = random.choice(route)
                route.remove(customer_to_replace)
                route.append(missing_customers.pop())

    return individual

# Setting up the Genetic Algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

def create_detailed_route_table(best_individual, problem, execution):
    route_data = []
    vehicles = 0
    visited_customers = []
    total_distance = 0

    for vehicle_idx, vehicle_route in enumerate(best_individual):
        current_time = 0
        last_customer = 0  # Start at depot
        accumulated_cost = 0
        if not vehicle_route:
          continue
        vehicles += 1

        for customer in vehicle_route:
            visited_customers.append(customer)
            travel_starts_at = current_time
            distance = calculate_distance(data.loc[last_customer, 'X'], data.loc[last_customer, 'Y'], data.loc[customer, 'X'], data.loc[customer, 'Y'])
            total_distance += distance
            arrival_time = travel_starts_at + distance
            ready_time = data.loc[customer, 'READY_TIME']
            due_time = data.loc[customer, 'DUE_TIME']
            service_time = data.loc[customer, 'SERVICE_TIME']
            waiting_time = max(0, ready_time - arrival_time)
            delay = max(0, arrival_time - due_time)
            ends_delivery_at = max(arrival_time, ready_time) + service_time

            # Acumulación del costo
            cost_for_leg = distance + service_time
            accumulated_cost += cost_for_leg

            route_data.append({
                'Vehicle': vehicle_idx + 1,
                'From Customer': f"{last_customer}: ({data.loc[last_customer, 'X']},  {data.loc[last_customer, 'Y']})",
                'To Customer': f"{customer}: ({data.loc[customer, 'X']},  {data.loc[customer, 'Y']})",
                'Travel Starts At': travel_starts_at,
                'Arrival Time': arrival_time,
                'Distance': distance,
                'Ready Time': ready_time,
                'Due Time': due_time,
                'Service Time': service_time,
                'Waiting Time': waiting_time,
                'Delay': delay,
                'Ends Delivery At': ends_delivery_at,
                'Accumulated Cost': accumulated_cost,
                'Accumulated Distance': round(total_distance, 2) # Añadir coste acumulado
            })

            current_time = ends_delivery_at
            last_customer = customer

        # Distance back to depot
        distance_back = calculate_distance(data.loc[last_customer, 'X'], data.loc[last_customer, 'Y'], data.loc[0, 'X'], data.loc[0, 'Y'])
        total_distance += distance_back
        arrival_time_back = current_time + distance_back
        accumulated_cost += distance_back

        route_data.append({
            'Vehicle': vehicle_idx + 1,
            'From Customer': f"{last_customer}: ({data.loc[last_customer, 'X']},  {data.loc[last_customer, 'Y']})",
            'To Customer': 0,  # Depot
            'Travel Starts At': current_time,
            'Arrival Time': arrival_time_back,
            'Distance': distance_back,
            'Ready Time': 0,
            'Due Time': 0,
            'Service Time': 0,
            'Waiting Time': 0,
            'Delay': 0,
            'Ends Delivery At': arrival_time_back,
            'Accumulated Cost': accumulated_cost,
            'Accumulated Distance': round(total_distance, 2) # Añadir coste acumulado
        })

    # Convert to DataFrame
    route_df = pd.DataFrame(route_data)
    route_df.to_csv(f"GA_{problem}_execution_{execution}.csv", index=False)
    total_wait_delay = route_df['Waiting Time'].sum() + route_df['Delay'].sum()
    return route_df, total_distance, total_wait_delay, vehicles, len(set(visited_customers))


def main(problem, execution):
    population = toolbox.population(n=100)
    halloffame = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    # Evaluate the entire population
    for individual in population:
        individual.fitness.values = evaluate(individual)

    algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.01, ngen=100, stats=stats, halloffame=halloffame, verbose=False)

    best_individual = halloffame[0]
    best_cost = best_individual.fitness.values[0]

    #plot_routes(best_individual)
    # Después de obtener la mejor solución con el algoritmo genético
    best_route_table, total_distance, total_wait_delay, vehicles, visited_customers = create_detailed_route_table(best_individual, problem, execution)

    print(f"Best score: {round(best_cost, 2)}")
    print(f"Total distance: {round(total_distance, 2)}")
    print(f"Waiting/delay cost = {round(total_wait_delay * WAIT_DELAY_COST, 2)}")
    print(f"Vehicles cost = {vehicles * VEHICLES_COST}")
    print(f"Unattended Cost:{(N_CUSTOMERS - visited_customers) * UNATTENDED_CUSTOMER_COST} ")
    print(f"Best Routes: {best_individual}")
    print(best_route_table)
    print("\n")

    return round(best_cost, 2), round(total_distance, 2), round(total_wait_delay * WAIT_DELAY_COST, 2), vehicles, (N_CUSTOMERS - visited_customers) * UNATTENDED_CUSTOMER_COST


def plot_routes(routes):
    plt.figure(figsize=(12, 10))

    # Color map for vehicles
    colors = plt.cm.jet(np.linspace(0, 1, len(routes)))

    for i, vehicle_route in enumerate(routes):
        x_coords = [data.loc[0, 'X']]  # Start from depot
        y_coords = [data.loc[0, 'Y']]
        for customer in vehicle_route:
            x_coords.append(data.loc[customer, 'X'])
            y_coords.append(data.loc[customer, 'Y'])
        x_coords.append(data.loc[0, 'X'])  # Return to depot
        y_coords.append(data.loc[0, 'Y'])

        # Plot the route
        plt.plot(x_coords, y_coords, marker='o', color=colors[i], label=f'Vehicle {i + 1}')

        # Add arrows to indicate direction
        for j in range(len(x_coords) - 1):
            plt.arrow(x_coords[j], y_coords[j], x_coords[j + 1] - x_coords[j], y_coords[j + 1] - y_coords[j],
                      head_width=0.3, head_length=0.3, fc=colors[i], ec=colors[i], alpha=0.6)

    # Plot depot and customers
    plt.scatter(data['X'][0], data['Y'][0], c='red', s=200, label='Depot')
    plt.scatter(data['X'][1:], data['Y'][1:], c='blue', label='Customers')

    # Annotate customers with their indices
    for i in range(1, len(data)):
        plt.text(data.loc[i, 'X'], data.loc[i, 'Y'], str(i), fontsize=12, ha='right')

    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Vehicle Routes')
    plt.legend()
    plt.grid()
    plt.show()


NUM_EXECUTIONS = 30
PROBLEM_LIST = [
    'vrptw_10_customers_problem_1.csv',
    'vrptw_10_customers_problem_3.csv',
    'vrptw_10_customers_problem_4.csv',
    'vrptw_10_customers_problem_5.csv',
    'vrptw_10_customers_problem_8.csv',
    'vrptw_10_customers_problem_9.csv',
    'vrptw_20_customers_problem_1.csv',
    'vrptw_20_customers_problem_3.csv',
    'vrptw_20_customers_problem_5.csv',
    'vrptw_20_customers_problem_8.csv',
    'vrptw_20_customers_problem_9.csv',
    'vrptw_20_customers_problem_10.csv',
]
PROBLEM_LIST = PROBLEM_LIST[6:12]

if __name__ == '__main__':
    # Ejecución de los problemas
    for problem in PROBLEM_LIST:
      results = []
      N_CUSTOMERS = 10 if "_10_" in problem else 20
      N_VEHICLES = 5 if "_10_" in problem else 10
      print(f"Running problem {problem}")
      for i in range(1, NUM_EXECUTIONS + 1):
        problem_start_time = time.time()
        # Cargar los datos
        global data
        data = pd.read_csv(f'./{problem}')
        # Ejecución del ACO
        cost, distance_cost, wait_delay_cost, vehicles_cost, unattended_cost = main(problem, i)
        end_time = time.time()
        elapsed_time = end_time - problem_start_time
        print(f"Execution {i} - time: {elapsed_time:.2f} seconds")
        print("\n")
        results.append((i,
                        problem,
                        cost,
                        distance_cost,
                        wait_delay_cost,
                        vehicles_cost * VEHICLES_COST,
                        unattended_cost,
                        elapsed_time,
                        N_VEHICLES))

      COLUMNS = ['Execution', 'Problem', 'Cost', 'Distamce', 'Wait_Delay_Cost', 'Vehicles_Cost', 'Unattended_Cost', 'Time', 'N_Vehicles']
      df = pd.DataFrame(results, columns = COLUMNS)
      df.to_csv(f'./GA_results_{problem}', index=False)
